In [1]:
!pip3 install darli mujoco robot_descriptions pin

In [2]:
import mujoco
import numpy as np
from darli.backend import CasadiBackend, PinocchioBackend
from darli.modeling import Functional, Robot
from robot_descriptions.skydio_x2_description import URDF_PATH
from robot_descriptions.skydio_x2_mj_description import MJCF_PATH

np.set_printoptions(precision=6, suppress=True)
np.random.seed(0)

In [3]:
mjmodel = mujoco.MjModel.from_xml_path(MJCF_PATH)
mjdata = mujoco.MjData(mjmodel)

mjdata.qpos

In [5]:
for act_id in range(4):
    mjmodel.actuator(act_id).ctrlrange = np.array([-100, 100])

original_q = mjdata.qpos.copy()
u = np.random.uniform(0, 1, 4)

mjdata.ctrl = u
mujoco.mj_step(mjmodel, mjdata)

mjdata.qacc

array([-0.034602,  0.059678, -7.989081,  1.107806,  0.641039,  0.112918])

## Check control and generalized force from MuJoCo

In [8]:
u, mjdata.qfrc_actuator

(array([0.548814, 0.715189, 0.602763, 0.544883]),
 array([ 0.000077, -0.000134,  2.411649,  0.040366,  0.01629 ,  0.004508]))

## Load model into Darli / (same can be done with pinocchio)

In [9]:
darli_model = Functional(CasadiBackend(URDF_PATH))

# length from center of mass to rotor
rotors_to_com = np.array(
    [[-0.14, -0.18, 0.05], [-0.14, 0.18, 0.05], [0.14, 0.18, 0.08], [0.14, -0.18, 0.08]]
)

kt = 1.0
km = 0.0201  # from gear ratio in MJCF
moment_selector = np.zeros((3, 4))

for i in range(4):
    moment_selector[:, i] = np.cross(rotors_to_com[i], np.array([0, 0, kt]))

moment_selector += np.array([[0, 0, 0, 0], [0, 0, 0, 0], [-km, km, km, -km]])

selector = np.vstack(
    [
        np.array([[0, 0, 0, 0], [0, 0, 0, 0], [1, 1, 1, 1]]),
        moment_selector,
    ]
)

darli_model.update_selector(selector)
selector

array([[ 0.    ,  0.    ,  0.    ,  0.    ],
       [ 0.    ,  0.    ,  0.    ,  0.    ],
       [ 1.    ,  1.    ,  1.    ,  1.    ],
       [-0.18  ,  0.18  ,  0.18  , -0.18  ],
       [ 0.14  ,  0.14  , -0.14  , -0.14  ],
       [-0.0201,  0.0201,  0.0201, -0.0201]])

## Compare the result of generalized force from MuJoCo and Darli

In [10]:
selector @ u

array([0.      , 0.      , 2.411649, 0.040366, 0.01629 , 0.004508])

## Compare the result of forward dynamics in MuJoCo and Darli

In [11]:
pinq = original_q
# scalar first to scalar last
pinq[[3, 4, 5, 6]] = pinq[[4, 5, 6, 3]]

darli_model.forward_dynamics(pinq, np.zeros(6), mjdata.ctrl)

DM([-0.0345919, 0.05978, -7.98989, 1.10781, 0.641039, 0.112906])